In [1]:
import cv2
import mediapipe as mp
import random
import time
#TODO: several moles at once, moles should expire after a certain time, track missed moles, restart button

# Function to draw hand landmarks
def draw_landmarks(frame, landmarks):
    for landmark in landmarks:
        x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

# Function to check collision between two rectangles
def is_collision(rect1, rect2):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2

    if x1 < x2 + w2 and x1 + w1 > x2 and y1 < y2 + h2 and y1 + h1 > y2:
        return True
    return False

# Initialize a new mole
def set_mole(frame):
    mole_size = 50
    mole_x = random.randint(0, frame.shape[1] - mole_size)
    mole_y = random.randint(0, frame.shape[0] - mole_size)

    return mole_x, mole_y, mole_x + mole_size, mole_y + mole_size


# Function to draw a mole at a random location
def draw_mole(frame, mole):
    mole_color = (0, 0, 255)
    a, b, c, d = mole
    #rec = cv2.rectangle(frame, (mole_x, mole_y), (mole_x + mole_size, mole_y + mole_size), mole_color, -1)
    cv2.rectangle(frame, (a, b), (c, d), mole_color, -1)

# Main function
def main():
    cap = cv2.VideoCapture(0)

    # Initialize Mediapipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    start_time = time.time()
    game_duration = 30  # Set the duration of the game in seconds

    mole = None
    score = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with Mediapipe Hands
        results = hands.process(rgb_frame)

        if time.time() - start_time < game_duration:
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Draw hand landmarks
                    draw_landmarks(frame, hand_landmarks.landmark)

                    # Get the coordinates of the index finger tip
                    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    x, y = int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])

                    # Check for collision with the mole
                    if mole and is_collision(mole, (x, y, 5, 5)):
                        mole = None
                        score += 1

            # Draw the mole if it doesn't exist
            if mole is None:
                mole = set_mole(frame)
                print(type(mole))

            draw_mole(frame,mole)

            # Display the score
            cv2.putText(frame, f"Score: {score}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Display the timer
            remaining_time = max(0, int(game_duration - (time.time() - start_time)))
            cv2.putText(frame, f"Time: {remaining_time}s", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        else:
            # Game over
            cv2.putText(frame, "Game Over", (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)


        # Display the frame
        cv2.imshow("Whack-a-Mole", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

2024-01-06 14:42:30.577 python[58131:1742945] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1704548551.070101       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


KeyboardInterrupt: 